In [235]:
from pyspark.sql import SparkSession
import pyspark.sql.types as types
import pyspark.sql.functions as F

spark = SparkSession.builder.appName("SparkSQL").getOrCreate()
sc = spark.sparkContext
spark

In [240]:
df = spark.read.csv("data/IBM.csv", header=True, inferSchema=True).sample(False, 0.3)

In [241]:
df.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Attrition: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- DistanceFromHome: integer (nullable = true)
 |-- Education: integer (nullable = true)
 |-- EducationField: string (nullable = true)
 |-- EnvironmentSatisfaction: integer (nullable = true)
 |-- JobSatisfaction: integer (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- MonthlyIncome: integer (nullable = true)
 |-- NumCompaniesWorked: integer (nullable = true)
 |-- WorkLifeBalance: integer (nullable = true)
 |-- YearsAtCompany: integer (nullable = true)



# calculating simple statistics on dataframe

In [352]:
df.describe().show()

df.select("Age", "MonthlyIncome").summary("count", "min", "25%", "75%", "max").show()

df.agg(F.kurtosis("Age"), F.skewness("Age")).show()

+-------+-----------------+---------+---------------+-----------------+------------------+----------------+-----------------------+-----------------+-------------+-----------------+------------------+------------------+-----------------+
|summary|              Age|Attrition|     Department| DistanceFromHome|         Education|  EducationField|EnvironmentSatisfaction|  JobSatisfaction|MaritalStatus|    MonthlyIncome|NumCompaniesWorked|   WorkLifeBalance|   YearsAtCompany|
+-------+-----------------+---------+---------------+-----------------+------------------+----------------+-----------------------+-----------------+-------------+-----------------+------------------+------------------+-----------------+
|  count|              440|      440|            440|              440|               440|             440|                    440|              440|          440|              440|               440|               440|              440|
|   mean|36.85227272727273|     null|           

# aggregation on dataframe

In [353]:
aggregations = [
    F.min("Age").alias("MinAge"),
    F.max("Age").alias("MaxAge"),
    F.stddev("Age").alias("StdDevAge"),
    F.avg("Age").alias("AvgAge"),
]

df.agg(*aggregations).show()


df.groupBy("Department","EducationField").count().orderBy("Department").show()

+------+------+-----------------+-----------------+
|MinAge|MaxAge|        StdDevAge|           AvgAge|
+------+------+-----------------+-----------------+
|    18|    60|9.327671831348484|36.85227272727273|
+------+------+-----------------+-----------------+

+--------------------+----------------+-----+
|          Department|  EducationField|count|
+--------------------+----------------+-----+
|     Human Resources|Technical Degree|    3|
|     Human Resources| Human Resources|    8|
|     Human Resources|         Medical|    2|
|     Human Resources|   Life Sciences|    4|
|Research & Develo...|           Other|   19|
|Research & Develo...|Technical Degree|   26|
|Research & Develo...|   Life Sciences|  138|
|Research & Develo...|         Medical|   93|
|               Sales|Technical Degree|   15|
|               Sales|         Medical|   26|
|               Sales|       Marketing|   45|
|               Sales|   Life Sciences|   59|
|               Sales|           Other|    2|
+--

# Selecting data from dataframe

In [354]:
# using sql to query dataframe
df.createOrReplaceTempView("sample_df_view")

sqlquery = """
SELECT
    DISTINCT Department, EducationField,
    MIN(MonthlyIncome) OVER (PARTITION BY Department, EducationField) AS MinMonthlyIncome,
    MAX(MonthlyIncome) OVER (PARTITION BY Department, EducationField) AS MaxMonthlyIncome,
    AVG(MonthlyIncome) OVER (PARTITION BY Department, EducationField) AS AvgAMonthlyIncome
FROM sample_df_view
ORDER BY Department , EducationField
"""
spark.sql(sqlquery).show()

+--------------------+----------------+----------------+----------------+------------------+
|          Department|  EducationField|MinMonthlyIncome|MaxMonthlyIncome| AvgAMonthlyIncome|
+--------------------+----------------+----------------+----------------+------------------+
|     Human Resources| Human Resources|            2187|           19141|          8044.125|
|     Human Resources|   Life Sciences|            2064|           14026|           5443.25|
|     Human Resources|         Medical|            2267|           10725|            6496.0|
|     Human Resources|Technical Degree|            2564|            4323|3194.3333333333335|
|Research & Develo...|   Life Sciences|            1393|           19740| 6095.442028985507|
|Research & Develo...|         Medical|            1706|           19701| 6803.193548387097|
|Research & Develo...|           Other|            2544|           19202| 6762.315789473684|
|Research & Develo...|Technical Degree|            1274|           192

In [355]:
# use select or drop to remove columns
df.select("Department","MaritalStatus").distinct()\
    .orderBy("Department","MaritalStatus").show()

+--------------------+-------------+
|          Department|MaritalStatus|
+--------------------+-------------+
|     Human Resources|     Divorced|
|     Human Resources|      Married|
|     Human Resources|       Single|
|Research & Develo...|     Divorced|
|Research & Develo...|      Married|
|Research & Develo...|       Single|
|               Sales|     Divorced|
|               Sales|      Married|
|               Sales|       Single|
+--------------------+-------------+



# joining two dataframes

In [264]:
education = spark.createDataFrame(
    sc.parallelize([
        (1, "Primary"),
        (2, "Secondary"),
        (3, "Tertiary"),
        (4, "PostGraduate"),
        (5, "Doctorate")
        ]),
    schema=["Education", "Desc"])

df.join(education, 
        on = ["Education"],
        how ="left").show(5)

+---------+---+---------+--------------------+----------------+----------------+-----------------------+---------------+-------------+-------------+------------------+---------------+--------------+---------+
|Education|Age|Attrition|          Department|DistanceFromHome|  EducationField|EnvironmentSatisfaction|JobSatisfaction|MaritalStatus|MonthlyIncome|NumCompaniesWorked|WorkLifeBalance|YearsAtCompany|     Desc|
+---------+---+---------+--------------------+----------------+----------------+-----------------------+---------------+-------------+-------------+------------------+---------------+--------------+---------+
|        1| 31|       No|Research & Develo...|              26|   Life Sciences|                      1|              3|     Divorced|         2911|                 1|              2|             5|  Primary|
|        1| 32|      Yes|Research & Develo...|              16|   Life Sciences|                      2|              1|       Single|         3919|                

# Renaming columns

In [328]:
# using select + functions.col and alias
df.select(
    F.col("Age").alias("NewAge"),
    F.col("MonthlyIncome").alias("NewMonthlyIncome")).show(3)

# using withColumnRenamed
df.withColumnRenamed("Age","NewAge").withColumnRenamed("MonthlyIncome","NewMonthlyIncome").show(3)

+------+----------------+
|NewAge|NewMonthlyIncome|
+------+----------------+
|    38|            2406|
|    59|            5473|
|    32|            6220|
+------+----------------+
only showing top 3 rows

+------+---------+--------------------+----------------+---------+--------------+-----------------------+---------------+-------------+----------------+------------------+---------------+--------------+
|NewAge|Attrition|          Department|DistanceFromHome|Education|EducationField|EnvironmentSatisfaction|JobSatisfaction|MaritalStatus|NewMonthlyIncome|NumCompaniesWorked|WorkLifeBalance|YearsAtCompany|
+------+---------+--------------------+----------------+---------+--------------+-----------------------+---------------+-------------+----------------+------------------+---------------+--------------+
|    38|       No|Research & Develo...|              29|        5| Life Sciences|                      4|              4|       Single|            2406|                 1|             

# filtering data

In [344]:
# using where
df.where((F.col("Age") > 40) & (F.col("MonthlyIncome")>10000)).show(2)

#using filter with SQL expression
df.filter('Department IN ("Sales","Human Resources")').show(2)

+---+---------+---------------+----------------+---------+---------------+-----------------------+---------------+-------------+-------------+------------------+---------------+--------------+
|Age|Attrition|     Department|DistanceFromHome|Education| EducationField|EnvironmentSatisfaction|JobSatisfaction|MaritalStatus|MonthlyIncome|NumCompaniesWorked|WorkLifeBalance|YearsAtCompany|
+---+---------+---------------+----------------+---------+---------------+-----------------------+---------------+-------------+-------------+------------------+---------------+--------------+
| 55|       No|          Sales|               1|        2|  Life Sciences|                      1|              4|      Married|        10239|                 3|              3|             1|
| 59|       No|Human Resources|               2|        4|Human Resources|                      3|              4|      Married|        18844|                 9|              3|             3|
+---+---------+---------------+----

# Transforming data

In [323]:
# df  = df.rdd.repartition(4).toDF()

# removing duplicates from dataframe

In [295]:
# use select or drop to remove columns

df.select("Department","MaritalStatus").distinct()\
    .orderBy("Department","MaritalStatus").show()

+--------------------+-------------+
|          Department|MaritalStatus|
+--------------------+-------------+
|     Human Resources|     Divorced|
|     Human Resources|      Married|
|     Human Resources|       Single|
|Research & Develo...|     Divorced|
|Research & Develo...|      Married|
|Research & Develo...|       Single|
|               Sales|     Divorced|
|               Sales|      Married|
|               Sales|       Single|
+--------------------+-------------+



In [288]:
sc.parallelize(range(10), 4).collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [327]:
df.toPandas().head(3)

,Age,Attrition,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,JobSatisfaction,MaritalStatus,MonthlyIncome,NumCompaniesWorked,WorkLifeBalance,YearsAtCompany
0,38,No,Research & Development,29,5,Life Sciences,4,4,Single,2406,1,3,10
1,59,No,Sales,1,1,Life Sciences,1,3,Single,5473,7,2,4
2,32,No,Research & Development,1,3,Life Sciences,2,2,Married,6220,1,3,10


In [140]:
%%sh

pip list

Package                         Version
------------------------------- ---------
apache-beam                     2.48.0
appdirs                         1.4.4
appnope                         0.1.3
asttokens                       2.2.1
backcall                        0.2.0
backports.functools-lru-cache   1.6.4
blis                            0.7.7
Bottleneck                      1.3.5
brotlipy                        0.7.0
cachetools                      5.3.1
catalogue                       2.0.7
certifi                         2022.12.7
cffi                            1.15.1
charset-normalizer              2.0.4
click                           8.0.4
cloudpickle                     2.2.1
colorama                        0.4.6
comm                            0.1.2
contourpy                       1.0.5
crcmod                          1.7
cryptography                    39.0.1
cycler                          0.11.0
cymem                           2.0.6
debugpy                         1.6.6


In [153]:
spark.stop()

In [183]:
# rdd.getNumPartitions() # num of partitions
# rdd.repartition(8) # change num of partitions
# rdd.coalesce(1) # change num of partitions without shuffling